# Day 4 - Lab 1: Automated Testing & Quality Assurance

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 4! An application without tests is an application that is broken by design. Today, we focus on quality assurance. You will act as a QA Engineer, using an AI co-pilot to build a robust test suite for the API you created yesterday. This is a critical step to ensure our application is reliable and ready for production.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the source code for our main application from `app/main.py`. Providing the full code as context is essential for the LLM to generate accurate and relevant tests.

**Model Selection:**
For generating tests, models with strong code understanding and logical reasoning are best. `gpt-4.1`, `o3`, `codex-mini`, and `gemini-2.5-pro` are all excellent choices for this task.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated test files.
- `clean_llm_output()`: To clean up the generated Python code.

In [2]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the application code from Day 3 to provide context for test generation
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

2025-10-31 09:39:59,619 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Task:** Generate basic `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

**Instructions:**
1.  Create a prompt that asks the LLM to act as a QA Engineer.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
4.  Generate another test for the `GET /users/` endpoint.
5.  Save the generated tests into a file named `tests/test_main_simple.py`.

**Expected Quality:** A Python script containing valid `pytest` functions that test the basic, successful operation of your API.

In [5]:
# TODO: Write a prompt to generate happy path tests for your API.
happy_path_tests_prompt = f"""
Act as a QA Engineer.
The current application code is as follows:
{app_code}
Generate a pytest test function for the POST /users/ endpoint, asserting that a user is created successfully,
checking for a 201 status code and correct response body.
Generate another test for the GET /users/ endpoint, asserting that it returns a list of users with a 200 status code.
Ensure that the tests are well-structured, use appropriate pytest fixtures if necessary, and include comments explaining each step.
Don't include unnecessary comments or explanations outside the code block.
"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple.py", overwrite=True)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Happy Path Tests ---
# tests/test_main.py

import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Assuming the application code is in a file named 'main.py' inside an 'app' directory
# and the db_models are in 'app/db_models.py'
from app.main import app, get_db
from app.db_models import Base

# --- Test Database Setup ---
# Use an in-memory SQLite database for testing to ensure tests are isolated and fast.
# `connect_args` is needed for SQLite to allow multi-threaded access.
SQLALCHEMY_DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)


# --- Dependency Override ---
# This function will override the `get_db` dependency in the main app.
# Instead of connecting to the production database, requests will use this
# in-memory test d

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Task:** Prompt the LLM to generate tests for common edge cases, such as providing invalid data or requesting a non-existent resource.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to write two new test functions:
    * A test for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.
    * A test for the `GET /users/{user_id}` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

**Expected Quality:** Two new `pytest` functions that verify the application handles common error scenarios correctly.

In [6]:
# TODO: Write a prompt to generate edge case tests.
edge_case_tests_prompt = f"""
Utilizing the same application code:
{app_code}
Generate one pytest test function for the POST /users/ endpoint that tries to create a user with an email that already exists, asserting that the API returns a 400 status code and an appropriate error message.
Generate another test for the GET /users/user_id endpoint that requests a non-existent user ID, asserting that the API returns a 404 Not Found error.
Ensure that the tests are well-structured, use appropriate pytest fixtures if necessary, and include comments explaining each step.
Don't include unnecessary comments or explanations outside the code block.
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Edge Case Tests ---
import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import StaticPool

# Assuming the provided application code is in a file named `main.py`
# and the SQLAlchemy models are in `db_models.py`
from main import app, get_db, Base, User

# --- Test Database Setup ---
# Use an in-memory SQLite database for testing to ensure tests are fast and isolated.
# `connect_args={"check_same_thread": False}` is needed for SQLite.
# `StaticPool` is used to ensure the same connection is used for the lifespan of the test session.
SQLALCHEMY_DATABASE_URL = "sqlite:///:memory:"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
    connect_args={"check_same_thread": False},
    poolclass=StaticPool,
)
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)


# --- Dependency Override ---
# This function will override the `get_db` dependency in

### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Task:** Generate a `pytest` fixture that creates a fresh, isolated, in-memory database for each test session. Then, refactor your tests to use this fixture. This is a critical pattern for professional-grade testing.

> **Hint:** Why use an isolated database? Running tests against your actual development database can lead to data corruption and flaky, unreliable tests. A pytest fixture that creates a fresh, in-memory database for each test ensures that your tests are independent, repeatable, and have no side effects.

**Instructions:**
1.  Create a prompt that asks the LLM to generate a `pytest` fixture.
2.  This fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
3.  It needs to create all the database tables before the test runs and tear them down afterward.
4.  Crucially, it must override the `get_db` dependency in your FastAPI app to use this temporary database during tests.
5.  Save the generated fixture code to a special file named `tests/conftest.py`.
6.  Finally, create a new test file, `tests/test_main_with_fixture.py`, and ask the LLM to rewrite the happy-path tests from Challenge 1 to use the new database fixture.

**Expected Quality:** Two new files, `tests/conftest.py` and `tests/test_main_with_fixture.py`, containing a professional `pytest` fixture for database isolation and tests that are correctly refactored to use it.

In [ ]:
# TODO: Write a prompt to generate the pytest fixture for an isolated test database.
db_fixture_prompt = f"""
Generate a pytest fixture.
The fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
It needs to create all the database tables before the test runs and tear them down afterward.
Crucially, it must override the get_db dependency in 
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py")
else:
    print("Skipping fixture generation because app context is missing.")

# TODO: Write a prompt to refactor the happy path tests to use the new fixture.
refactor_tests_prompt = f"""
# Your prompt here
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py")
else:
    print("Skipping test refactoring because app context is missing.")

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.